# AutoGen ప్రాథమిక నమూనా

ఈ కోడ్ నమూనాలో, మీరు [AutoGen](https://aka.ms/ai-agents/autogen) AI ఫ్రేమ్‌వర్క్‌ను ఉపయోగించి ఒక ప్రాథమిక ఏజెంట్‌ను సృష్టిస్తారు.

ఈ నమూనా లక్ష్యం భిన్నమైన ఏజెంటిక్ నమూనాలను అమలు చేసే అదనపు కోడ్ నమూనాల్లో మేము ఉపయోగించే దశలను మీకు చూపించడం.


## అవసరమైన పైథాన్ ప్యాకేజీలను దిగుమతి చేసుకోండి


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## క్లయింట్‌ను సృష్టించండి

ఈ ఉదాహరణలో, LLM యాక్సెస్ కోసం [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ఉపయోగిస్తాము.

`model` ను `gpt-4o-mini` గా నిర్వచించారు. GitHub Models మార్కెట్‌ప్లేస్‌లో అందుబాటులో ఉన్న మరో మోడల్‌కు మోడల్‌ను మార్చి వివిధ ఫలితాలను చూడండి.

త్వరిత పరీక్షగా, మేము ఒక సింపుల్ ప్రాంప్ట్‌ను నడుపుతాము - `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ఏజెంట్‌ను నిర్వచించడం

మేము `client`ను సెటప్ చేసి, అది పనిచేస్తుందని నిర్ధారించిన తర్వాత, ఇప్పుడు `AssistantAgent`ను సృష్టిద్దాం. ప్రతి ఏజెంట్‌కు కేటాయించవచ్చు:  
**name** - మల్టీ-ఏజెంట్ ఫ్లోలలో దీనిని సూచించడానికి ఉపయోగపడే సంక్షిప్త పేరు.  
**model_client** - మీరు ముందటి దశలో సృష్టించిన క్లయింట్.  
**tools** - ఏజెంట్ ఒక పని పూర్తి చేయడానికి ఉపయోగించగల సాధనాలు.  
**system_message** - LLM యొక్క పని, ప్రవర్తన మరియు టోన్‌ను నిర్వచించే మెటాప్రాంప్ట్.  

LLM ఎలా స్పందిస్తుందో చూడటానికి మీరు సిస్టమ్ మెసేజ్‌ను మార్చవచ్చు. `tools` గురించి మేము పాఠం #4లో చర్చిస్తాము.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## ఏజెంట్‌ను నడపండి

క్రింది ఫంక్షన్ ఏజెంట్‌ను నడుపుతుంది. కొత్త సందేశంతో ఏజెంట్ యొక్క స్థితిని నవీకరించడానికి మేము `on_message` పద్ధతిని ఉపయోగిస్తాము.

ఈ సందర్భంలో, మేము వినియోగదారుని నుండి వచ్చిన కొత్త సందేశంతో స్థితిని నవీకరిస్తాము, అది `"Plan me a great sunny vacation"`.

మీరు సందేశం విషయాన్ని మార్చి LLM ఎలా భిన్నంగా స్పందిస్తుందో చూడవచ్చు.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**విమర్శ**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించారు. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తున్నప్పటికీ, ఆటోమేటెడ్ అనువాదాలలో తప్పులు లేదా అసమానతలు ఉండవచ్చు. దయచేసి, మూల భాషలో ఉన్న అసలు పత్రాన్ని అధికారం కలిగిన మూలంగా పరిగణించండి. ముఖ్యమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారులు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
